In [ ]:
from pathlib import Path 
import os 
os.chdir(Path("~/SecurityAnalytics/DataPreprocess").expanduser())

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [ ]:
VARIATION = "nocommentss"
jsonl_path = f"data/training_datasets/verified_{VARIATION}.jsonl"
#jsonl_path = "data/training_datasets/verified_nocomments.jsonl"

df = pd.read_json(jsonl_path, lines=True)

In [35]:
df.head()

,filename,filetype,content,variant
0,cryptol/examples/splitAt.cry,cry,"x = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([...",hybrid
1,cryptol/examples/AE.cry,cry,/*\nImplementation of the algorithms from the ...,hybrid
2,cryptol/examples/xor_cipher.cry,cry,encrypt : {a}(fin a) => [8] -> [a][8] -> [a][8...,hybrid
3,cryptol/examples/zero_weird.cry,cry,x : {a}() => a -> [16]\nx v = zero v \n\nprope...,hybrid
4,cryptol/examples/builtin_lifting.cry,cry,"//builtins lift over tuples, seqs, and records...",hybrid


In [ ]:
from src.preprocessing.interpreter_process import verify_df_row_with_cryptol
from src.preprocessing.saw_subprocess import run_saw_script, load_saw_results

saw_results = []
MOUNT_DIR = "/Users/josh/SecurityAnalytics/DataPreprocess/cryptol-files"
SERVER_URL = "http://localhost:8080"
print(verify_df_row_with_cryptol(df, 35, host_mount_dir=MOUNT_DIR, server_url=SERVER_URL))
rows = []
results_df = load_saw_results("data/training_datasets/syntax_check/saw_results.jsonl")
for i in range(len(df)):
    if df.iloc[i]["filetype"] == 'cry':
        repl_result = verify_df_row_with_cryptol(
                df, i, host_mount_dir=MOUNT_DIR, server_url=SERVER_URL)
        rows.append({"filename" : df.iloc[i]["filename"], **repl_result["load_info"]})
    else:
        if df.iloc[i]["filename"] in results_df['filename'].values:
            result_ = results_df[results_df['filename'] == df.iloc[i]["filename"]]
            rows.append({"filename" : df.iloc[i]["filename"], "load_ok": result_["load_ok"].values[0], "error": result_["error"].values[0] if not result_["load_ok"].values[0] else None, "file_deps": ["SAW"]})
        else:
            saw_result = run_saw_script(f"/Users/josh/SecurityAnalytics/development/{df.iloc[i]['filename']}")
            results_df = pd.concat([results_df, pd.DataFrame({"filename": df.iloc[i]["filename"], **saw_result}, index=[0])], ignore_index=True)
            #saw_results.append({"filename": df.iloc[i]["filename"], **saw_result})
            if saw_result["returncode"] == 0:
                rows.append({"filename" : df.iloc[i]["filename"], "load_ok": True, "error": None, "file_deps": ["SAW"]})
            else:
                rows.append({"filename" : df.iloc[i]["filename"], "load_ok": False, "error": "saw", "file_deps": ["SAW"]})

{'host_path': '/Users/josh/SecurityAnalytics/DataPreprocess/cryptol-files/aes-mct-ecb_hxaq9fe6.cry', 'container_relpath': 'files/aes-mct-ecb_hxaq9fe6.cry', 'load_info': {'load_ok': True, 'file': 'files/aes-mct-ecb_hxaq9fe6.cry', 'error': 'None', 'file_deps': ['Cryptol', 'SuiteB']}}


In [37]:
results_df.head()

,filename,load_ok,returncode,stdout,stderr,cmd,cwd,error
0,cryptol-specs/Primitive/Symmetric/Cipher/Block...,False,2,"[01:25:14.935] Loading file ""/Users/josh/Secur...",,"[saw, /Users/josh/SecurityAnalytics/developmen...",/Users/josh/SecurityAnalytics/development/cryp...,NaN
1,cryptol-specs/Primitive/Symmetric/Cipher/Block...,True,0,"[01:25:15.332] Loading file ""/Users/josh/Secur...",,"[saw, /Users/josh/SecurityAnalytics/developmen...",/Users/josh/SecurityAnalytics/development/cryp...,NaN
2,saw-script/intTests/test1533/test.saw,True,0,"[01:25:21.098] Loading file ""/Users/josh/Secur...",,"[saw, /Users/josh/SecurityAnalytics/developmen...",/Users/josh/SecurityAnalytics/development/saw-...,NaN
3,saw-script/intTests/test_property_as_rewrite_r...,True,0,"[01:25:21.596] Loading file ""/Users/josh/Secur...",,"[saw, /Users/josh/SecurityAnalytics/developmen...",/Users/josh/SecurityAnalytics/development/saw-...,NaN
4,saw-script/intTests/test2148/test.saw,True,0,"[01:25:22.022] Loading file ""/Users/josh/Secur...",,"[saw, /Users/josh/SecurityAnalytics/developmen...",/Users/josh/SecurityAnalytics/development/saw-...,NaN


In [38]:
repl_results_df = pd.DataFrame(rows)
repl_results_df.head()

,filename,load_ok,file,error,file_deps
0,cryptol/examples/splitAt.cry,True,files/splitAt_svsvuxqv.cry,None,[Cryptol]
1,cryptol/examples/AE.cry,True,files/AE_gbs0cnqb.cry,None,error
2,cryptol/examples/xor_cipher.cry,True,files/xor_cipher_8segcox6.cry,None,[Cryptol]
3,cryptol/examples/zero_weird.cry,True,files/zero_weird_5mws3lbm.cry,None,[Cryptol]
4,cryptol/examples/builtin_lifting.cry,True,files/builtin_lifting_nf_r811j.cry,None,[Cryptol]


In [39]:
repl_results_df.to_json(f"data/training_datasets/syntax_check/{VARIATION}.jsonl", orient="records", lines=True)

In [40]:
print(len(results_df[results_df["load_ok"] == True]))
print(len(results_df[results_df["load_ok"] == False]))

223
168
